1. Import necessary libraries for data extraction, manipulation and loading

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#pd.set_option('display.max_colwidth', None)

#pd.set_option('display.width', 1000)
#pd.options.display.width = 0
#pd.set_option('display.expand_frame_repr', False)

import requests
import json
from pandas.io.json import json_normalize
#FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead

import zipfile, io #libraries to save zip files in project directory

url = ''#'https://api.fda.gov/animalandveterinary/event.json?skip=5&limit=10'
results = []
type(results)
#results=requests.get(url).json()['results']

list

2. Download file links and load them in a pandas dataframe.
It is noted that every time the data is updated you have to download all the files

In [2]:
######
#Download url for the required json files
downloadURL = 'https://api.thedogapi.com/v1/breeds'

####
#Get only the animal and veterinary data and load them in a pandas dataframe
dogs_df=pd.json_normalize(requests.get(downloadURL).json())

#df manipulations and sorting
#links_df.sort_values(by=['display_name'], inplace=True)
#links_df.reset_index(drop=True, inplace=True)
dogs_df

#dogs_df.to_csv('dog_breeds.csv', index=False, mode='w', header=True)

,id,name,bred_for,breed_group,life_span,temperament,origin,reference_image_id,weight.imperial,weight.metric,height.imperial,height.metric,image.id,image.width,image.height,image.url,country_code,description,history
0,1,Affenpinscher,"Small rodent hunting, lapdog",Toy,10 - 12 years,"Stubborn, Curious, Playful, Adventurous, Activ...","Germany, France",BJa4kxc4X,6 - 13,3 - 6,9 - 11.5,23 - 29,BJa4kxc4X,1600,1199,https://cdn2.thedogapi.com/images/BJa4kxc4X.jpg,NaN,NaN,NaN
1,2,Afghan Hound,Coursing and hunting,Hound,10 - 13 years,"Aloof, Clownish, Dignified, Independent, Happy","Afghanistan, Iran, Pakistan",hMyT4CDXR,50 - 60,23 - 27,25 - 27,64 - 69,hMyT4CDXR,606,380,https://cdn2.thedogapi.com/images/hMyT4CDXR.jpg,AG,NaN,NaN
2,3,African Hunting Dog,A wild pack animal,NaN,11 years,"Wild, Hardworking, Dutiful",,rkiByec47,44 - 66,20 - 30,30,76,rkiByec47,500,335,https://cdn2.thedogapi.com/images/rkiByec47.jpg,NaN,NaN,NaN
3,4,Airedale Terrier,"Badger, otter hunting",Terrier,10 - 13 years,"Outgoing, Friendly, Alert, Confident, Intellig...","United Kingdom, England",1-7cgoZSh,40 - 65,18 - 29,21 - 23,53 - 58,1-7cgoZSh,645,430,https://cdn2.thedogapi.com/images/1-7cgoZSh.jpg,NaN,NaN,NaN
4,5,Akbash Dog,Sheep guarding,Working,10 - 12 years,"Loyal, Independent, Intelligent, Brave",,26pHT3Qk7,90 - 120,41 - 54,28 - 34,71 - 86,26pHT3Qk7,600,471,https://cdn2.thedogapi.com/images/26pHT3Qk7.jpg,NaN,NaN,NaN
5,6,Akita,Hunting bears,Working,10 - 14 years,"Docile, Alert, Responsive, Dignified, Composed...",NaN,BFRYBufpm,65 - 115,29 - 52,24 - 28,61 - 71,BFRYBufpm,1280,853,https://cdn2.thedogapi.com/images/BFRYBufpm.jpg,NaN,NaN,NaN
6,7,Alapaha Blue Blood Bulldog,Guarding,Mixed,12 - 13 years,"Loving, Protective, Trainable, Dutiful, Respon...",NaN,33mJ-V3RX,55 - 90,25 - 41,18 - 24,46 - 61,33mJ-V3RX,1828,2065,https://cdn2.thedogapi.com/images/33mJ-V3RX.jpg,NaN,The Alapaha Blue Blood Bulldog is a well-devel...,
7,8,Alaskan Husky,Sled pulling,Mixed,10 - 13 years,"Friendly, Energetic, Loyal, Gentle, Confident",NaN,-HgpNnGXl,38 - 50,17 - 23,23 - 26,58 - 66,-HgpNnGXl,500,500,https://cdn2.thedogapi.com/images/-HgpNnGXl.jpg,NaN,NaN,NaN
8,9,Alaskan Malamute,"Hauling heavy freight, Sled pulling",Working,12 - 15 years,"Friendly, Affectionate, Devoted, Loyal, Dignif...",NaN,dW5UucTIW,65 - 100,29 - 45,23 - 25,58 - 64,dW5UucTIW,1023,769,https://cdn2.thedogapi.com/images/dW5UucTIW.jpg,NaN,NaN,NaN
9,10,American Bulldog,NaN,Working,10 - 12 years,"Friendly, Assertive, Energetic, Loyal, Gentle,...",NaN,pk1AAdloG,60 - 120,27 - 54,22 - 27,56 - 69,pk1AAdloG,1669,1377,https://cdn2.thedogapi.com/images/pk1AAdloG.jpg,NaN,NaN,NaN


In [3]:
import psycopg2
#import transformations
connection_string = 'postgresql://postgres:sa@localhost:5432/vets_dw'


#or this way with URI (Postgres 9.2 and above)
#uri = f'postgresql://{user}:{pass}@{host}:{port}/{database}'

#connection = psycopg2.connect(
#    database="vets_dw", 
#    user='postgres', 
#    password='sa', 
#    host='127.0.0.1', 
#    port= '5432')
connection = psycopg2.connect(connection_string)

connection.autocommit = True
cursor = connection.cursor()

cursor.execute('SELECT Version()')
version = cursor.fetchall()
print(version)

#create_table = """
#CREATE TABLE IF NOT EXISTS employee(
#   id SERIAL PRIMARY KEY ,
#    fullname TEXT,
#    email TEXT,
#    gender TEXT,
#    title TEXT,
#    salary NUMERIC
#)
#"""


insert_query = """
INSERT INTO staging.DogApiData (
 id,
 name,
 bred_for,
 breed_group,
 life_span,
 temperament,
 origin,
 reference_image_id,
 weight_imperial,
 weight_metric,
 height_imperial,
 height_metric,
 image_id,
 image_width,
 image_height,
 image_url,
 country_code,
 description,
 history)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

for index, breed in dogs_df.iterrows():
    cursor.execute(insert_query, breed)
        
cursor.execute('SELECT * FROM staging.DogApiData')

res = cursor.fetchall()
print(cursor.rowcount)
print(res)


cursor.close()
connection.close()




[('PostgreSQL 14.1, compiled by Visual C++ build 1914, 64-bit',)]
172


[(1, 'Affenpinscher', 'Small rodent hunting, lapdog', 'Toy', '10 - 12 years', 'Stubborn, Curious, Playful, Adventurous, Active, Fun-loving', 'Germany, France', 'BJa4kxc4X', '6 - 13', '3 - 6', '9 - 11.5', '23 - 29', 'BJa4kxc4X', 1600, 1199, 'https://cdn2.thedogapi.com/images/BJa4kxc4X.jpg', 'NaN', 'NaN', 'NaN'), (2, 'Afghan Hound', 'Coursing and hunting', 'Hound', '10 - 13 years', 'Aloof, Clownish, Dignified, Independent, Happy', 'Afghanistan, Iran, Pakistan', 'hMyT4CDXR', '50 - 60', '23 - 27', '25 - 27', '64 - 69', 'hMyT4CDXR', 606, 380, 'https://cdn2.thedogapi.com/images/hMyT4CDXR.jpg', 'AG', 'NaN', 'NaN'), (3, 'African Hunting Dog', 'A wild pack animal', 'NaN', '11 years', 'Wild, Hardworking, Dutiful', '', 'rkiByec47', '44 - 66', '20 - 30', '30', '76', 'rkiByec47', 500, 335, 'https://cdn2.thedogapi.com/images/rkiByec47.jpg', 'NaN', 'NaN', 'NaN'), (4, 'Airedale Terrier', 'Badger, otter hunting', 'Terrier', '10 - 13 years', 'Outgoing, Friendly, Alert, Confident, Intelligent, Courageous